In [18]:
from model_dnn 






    


In [15]:
model = tworz_folder('model_probny', 'pierwszy',
             [30], na_ilu_statystyki = 10000)

policzylo mean oraz variance
zapisalo je
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'model_probny/model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f14a71a2a20>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [20]:
dataset = input_fn('pierwszy','model_probny', 10,  
         True, 1, True, 10,
         True )

In [21]:
costam = dataset.make_one_shot_iterator()
sess = tf.InteractiveSession()
sess.run(costam.get_next())

/home/pawel/anaconda3/envs/root_tensorflow/lib/python3.6/site-packages/tensorflow/python/client/session.py:1711: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


({'n': array([[-0.04469895, -0.04469948, -0.04469915, -0.04469931, -0.0446989 ],
         [-0.04469895, -0.04469948, -0.04469915, -0.04469931, -0.0446989 ],
         [-0.04469895, -0.04469948, -0.04469915, -0.04469931, -0.0446989 ],
         [-0.04469895, -0.04469948, -0.04469915, -0.04469931, -0.0446989 ],
         [-0.04469895, -0.04469948, -0.04469915, -0.04469931, -0.0446989 ],
         [-0.04469895, -0.04469948, -0.04469915, -0.04469931, -0.0446989 ],
         [-0.04469895, -0.04469948, -0.04469915, -0.04469931, -0.0446989 ],
         [-0.04469895, -0.04469948, -0.04469915, -0.04469931, -0.0446989 ],
         [-0.04469895, -0.04469948, -0.04469915, -0.04469931, -0.0446989 ],
         [-0.04469895, -0.04469948, -0.04469915, -0.04469931, -0.0446989 ]],
        dtype=float32), 'c': array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5])},
 array([[0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0]]))